In [ ]:
%load_ext lab_black

In [ ]:
import json
import datetime as dt

import mplfinance as mpf
import pandas as pd
import numpy as np
import pandera as pa

import shared


logger = shared.get_logger()

In [ ]:
klines_btc_1m_df = shared.read_binance_klines()
klines_btc_1h_df = shared.split_df(df=klines_btc_1m_df, interval_str="1h")

In [ ]:
trades_btc_df = shared.read_binance_trades(
    date_start=dt.date.fromisoformat("2024-04-05"),
    date_end=dt.date.fromisoformat("2024-04-07"),
)

### weekends-pumps-dumps on 2024-03-30

In [ ]:
sundays_list = pd.date_range(start="2023-08-01", end="2024-03-30", freq="W").tolist()
weekeds_dt_ranges = [
    (sunday_dt - dt.timedelta(days=2), sunday_dt + dt.timedelta(days=1))
    for sunday_dt in sundays_list
]
dt_threshold = dt.datetime.fromisoformat("2023-08-01")
for dt_start, dt_end in weekeds_dt_ranges:
    slice_df = klines_btc_1h_df[lambda x: x.index >= dt_threshold][
        lambda x: x.index >= dt_start
    ][lambda x: x.index <= dt_end]
    fig, axlist = mpf.plot(
        slice_df,
        warn_too_much_data=len(slice_df),
        type="candle",
        figsize=(14, 3),
        style=shared.s,
        returnfig=True,
        vlines={
            "vlines": [(dt_start + dt.timedelta(days=1)).date()],
            "linewidths": 0.2,
        },
    )
    max_diff_abs = slice_df.high.max() - slice_df.low.min()
    max_diff_rel = (max_diff_abs / slice_df.low.min()) * 100
    x_label = f"[{dt_start.date()} - {dt_end.date()}] max_diff_rel={max_diff_rel:.2f}"
    axlist[-2].set_xlabel(x_label)
    axlist[-2].xaxis.set_ticklabels([])

### aprils on 2024-04-02

In [ ]:
for year in [2023, 2022, 2021, 2020, 2019, 2018]:
    dt_start = dt.datetime.fromisoformat(f"{year}-04-01")
    dt_end = dt.datetime.fromisoformat(f"{year}-05-01")
    slice_df = klines_btc_1h_df[lambda x: x.index >= dt_start][
        lambda x: x.index <= dt_end
    ]
    fig, axlist = mpf.plot(
        slice_df,
        warn_too_much_data=len(slice_df),
        type="candle",
        figsize=(14, 3),
        style=shared.s,
        returnfig=True,
    )
    max_diff_abs = slice_df.high.max() - slice_df.low.min()
    max_diff_rel = (max_diff_abs / slice_df.low.min()) * 100
    x_label = f"[{dt_start.date()} - {dt_end.date()}] max_diff_rel={max_diff_rel:.2f}"
    axlist[-2].set_xlabel(x_label)
    axlist[-2].xaxis.set_ticklabels([])

### ema-hit

In [ ]:
dt_start = dt.datetime.fromisoformat("2024-03-01")
dt_end = dt.datetime.fromisoformat("2024-03-31")
slice_df = klines_btc_1h_df[lambda x: x.index >= dt_start][lambda x: x.index <= dt_end]
highs_vals = []
highs_alines = []

for _, row in slice_df.iterrows():
    while highs_vals and highs_vals[-1] < row.high:
        highs_vals.pop()
        highs_alines.pop()
    ts = dt.datetime.fromtimestamp(row.open_time / 1000)
    aline = [(ts, row.high + 50), (ts, row.high + 100)]
    highs_vals.append(row.high)
    highs_alines.append(aline)
    assert len(highs_vals) == len(highs_alines)

mpf.plot(
    slice_df,
    warn_too_much_data=len(slice_df),
    alines={"alines": list(highs_alines), "colors": ["#f19d38"]},
    type="candle",
    figsize=(14, 4),
    style=shared.s,
    ema=120,
)

In [ ]:
slice_df = klines_btc_1h_df[
    lambda x: x.index >= dt.datetime.fromisoformat("2024-03-01")
].head(30)
t_indicator = mpf.make_addplot(slice_df[["high", "low"]])
fig, axlist = mpf.plot(
    slice_df,
    type="candle",
    figsize=(14, 4),
    style=shared.s,
    returnfig=True,
    addplot=t_indicator,
)
??mpf.make_addplot

### compare candles with binance candles

In [ ]:
# def calc_emas(prices: list[float], len_: int) -> list[float]:
#     smoothing = 2.0
#     len_sma = sum(prices[:len_]) / len_
#     emas = [len_sma] * len_
#     for i in range(len_, len(prices)):
#         k = smoothing / (1 + len_)
#         ema = prices[i] * k + emas[-1] * (1.0 - k)
#         emas.append(ema)
#     assert len(prices) == len(emas)
#     return emas


def calc_emas(prices: list[float], len_: int) -> list[float]:
    smoothing = 2.0
    emas = [prices[0]]
    for i in range(1, len(prices)):
        k = smoothing / (1 + len_)
        ema = prices[i] * k + emas[-1] * (1.0 - k)
        emas.append(ema)
    assert len(prices) == len(emas)
    return emas

In [ ]:
slice_df.close.ewm(com=2).mean()

In [ ]:
# binance data on 2024-04-06 00:00 is (67820.63, 68065.23, 67686.23, 67900.57)

dt_start = dt.datetime.fromisoformat("2024-04-02")
dt_end = dt.datetime.fromisoformat("2024-04-07")
slice_df = klines_btc_1h_df[lambda x: x.index >= dt_start][lambda x: x.index <= dt_end]

mpf.plot(
    slice_df,
    type="candle",
    figsize=(14, 4),
    style=shared.s,
    ema=12,
    addplot=mpf.make_addplot(
        # calc_emas(slice_df.close.tolist(), len_=12),
        # slice_df.close.ewm(span=12, adjust=False).mean(),
        emas_2024_04_08,
        color="orange",
        linestyle="dashed",
    ),
    vlines={
        "vlines": [dt.date.fromisoformat("2024-04-06")],
        "linewidths": 0.2,
    },
)

In [ ]:
with open("/home/jovyan/.var/close-prices-2024-04-08.json", "w") as f:
    f.write(json.dumps({"close_prices": slice_df.close.tolist()}))

In [ ]:
emas_2024_04_08 = json.load(open("/home/jovyan/.var/emas-2024-04-08.json"))["emas"]

In [ ]:
# emas_2024_04_08

### draw trade-emas backtest

In [ ]:
debug_candles_df = pd.read_sql_query(
    """
    select *
    from public.debug_candles;
""",
    con=shared.get_sa_engine(),
).set_index("open_time")
debug_emas_df = pd.read_sql_query(
    """
    select *
    from public.debug_emas;
""",
    con=shared.get_sa_engine(),
)
backtests_df = pd.read_sql_query(
    """
    select *
    from public.backtests;
""",
    con=shared.get_sa_engine(),
)
len(debug_candles_df), len(debug_emas_df), len(backtests_df)

In [ ]:
alines_list = []
for _, row in backtests_df.iterrows():
    if row.close_timestamp > debug_candles_df.index.max():
        continue
    b = [(row.open_timestamp, row.open_price), (row.close_timestamp, row.close_price)]
    alines_list.append(b)
emas = [
    80_000 for _ in range(len(debug_candles_df) - len(debug_emas_df))
] + debug_emas_df.value_ema.to_list()
mpf.plot(
    debug_candles_df,
    type="candle",
    figsize=(140, 40),
    style=shared.s,
    ema=12,
    addplot=mpf.make_addplot(emas, color="orange", linestyle="dashed"),
    alines={"alines": alines_list, "colors": ["r"], "linewidths": 5},
)